[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/open-atmos/PyPartMC/blob/main/examples/condense.ipynb)   
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PyPartMC/blob/main/examples/condense.ipynb)    
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PyPartMC.git/main?urlpath=lab/tree/examples/condense.ipynb)

In [1]:
# This file is a part of PyPartMC licensed under the GNU General Public License v3
# Copyright (C) 2022 University of Illinois Urbana-Champaign
# Based on (GPL v2 code): https://github.com/compdyn/partmc/tree/master/test/condense
# Authors:
#  - https://github.com/compdyn/partmc/graphs/contributors
#  - https://github.com/open-atmos/PyPartMC/graphs/contributors

In [2]:
import sys
if 'google.colab' in sys.modules:
    !pip --verbose install git+https://github.com/open-atmos/PyPartMC.git

In [3]:
import PyPartMC as ppmc
from PyPartMC import si

In [4]:
gas_data = ppmc.GasData(
    ("H2SO4", "HNO3", "HCl", "NH3", "NO", "NO2")
)

In [5]:
env_state = ppmc.EnvState({
    "rel_humidity": 0.95,
    "latitude": 0,
    "longitude": 0,
    "altitude": 0 * si.m,
    "start_time": 81000 * si.s,
    "start_day": 200
})

In [6]:
aero_data = ppmc.AeroData((
    #         density  ions in soln (1) molecular weight    kappa (1)
    #         |                     |   |                   |
    {"SO4":  [1800 * si.kg/si.m**3, 1,  96.0 * si.g/si.mol, 0.00]},
    {"NO3":  [1800 * si.kg/si.m**3, 1,  62.0 * si.g/si.mol, 0.00]},
    {"Cl":   [2200 * si.kg/si.m**3, 1,  35.5 * si.g/si.mol, 0.00]},
    {"NH4":  [1800 * si.kg/si.m**3, 1,  18.0 * si.g/si.mol, 0.00]},
    {"MSA":  [1800 * si.kg/si.m**3, 0,  95.0 * si.g/si.mol, 0.53]},
    {"ARO1": [1400 * si.kg/si.m**3, 0, 150.0 * si.g/si.mol, 0.10]},
    {"ARO2": [1400 * si.kg/si.m**3, 0, 150.0 * si.g/si.mol, 0.10]},
    {"ALK1": [1400 * si.kg/si.m**3, 0, 140.0 * si.g/si.mol, 0.10]},
    {"OLE1": [1400 * si.kg/si.m**3, 0, 140.0 * si.g/si.mol, 0.10]},
    {"API1": [1400 * si.kg/si.m**3, 0, 184.0 * si.g/si.mol, 0.10]},
    {"API2": [1400 * si.kg/si.m**3, 0, 184.0 * si.g/si.mol, 0.10]},
    {"LIM1": [1400 * si.kg/si.m**3, 0, 200.0 * si.g/si.mol, 0.10]},
    {"LIM2": [1400 * si.kg/si.m**3, 0, 200.0 * si.g/si.mol, 0.10]},
    {"CO3":  [2600 * si.kg/si.m**3, 1,  60.0 * si.g/si.mol, 0.00]},
    {"Na":   [2200 * si.kg/si.m**3, 1,  23.0 * si.g/si.mol, 0.00]},
    {"Ca":   [2600 * si.kg/si.m**3, 1,  40.0 * si.g/si.mol, 0.00]},
    {"OIN":  [2600 * si.kg/si.m**3, 0,   1.0 * si.g/si.mol, 0.10]},
    {"OC":   [1400 * si.kg/si.m**3, 0,   1.0 * si.g/si.mol, 0.10]},
    {"BC":   [1800 * si.kg/si.m**3, 0,   1.0 * si.g/si.mol, 0.00]},
    {"H2O":  [1000 * si.kg/si.m**3, 0,  18.0 * si.g/si.mol, 0.00]}
))

In [7]:
N_PART = 44
aero_state = ppmc.AeroState(N_PART, aero_data)

In [8]:
gas_state = ppmc.GasState()

In [9]:
times = [0 * si.s, 1200 * si.s]
null_gas = [{"time": [0 * si.s]}, {"rate": [1 / si.s]}]
null_aero = [
    {"time": [0 * si.s]},
    {"rate": [0 / si.s]},
    {"dist": {}}
]

scenario = ppmc.Scenario(gas_data, aero_data, {
    "temp_profile": [
        {"time": times},
        {"temp": [290 * si.K, 280 * si.K]}
    ],
    "pressure_profile": [
        {"time": times},
        {"pressure": [1000 * si.hPa, 800 * si.hPa]}
    ],
    "height_profile": [
        {"time": times},
        {"height": [200 * si.m, 200 * si.m]}
    ],
    "gas_emissions": null_gas,
    "gas_background": null_gas,
    "aero_emissions": null_aero,
    "aero_background": null_aero,
    "loss_function": "none"
})

In [13]:
run_part_opt = ppmc.RunPartOpt({
    "do_coagulation": False,
    "do_parallel": False,
    "t_max": 0 * si.s,
    "del_t": 1 * si.s,
    "t_output": 0,
    "t_progress": 0,
    "allow_halving": False,
    "allow_doubling": False
})

In [14]:
# ppmc.condense_equilib_particles(env_state, aero_data, aero_state)

In [15]:
ppmc.run_part(
    scenario, env_state, aero_data, aero_state,
    gas_data, gas_state, run_part_opt
)